In [1]:
from openai import OpenAI
from prompts import typo_style_aug_sys_prompt
from tqdm import tqdm

import os
import json
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [3]:
def typos_augment(queries: dict[str, list[str]]):
    res = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": typo_style_aug_sys_prompt},
            {"role": "user", "content": json.dumps(queries, indent=4)},
        ],
        response_format={"type": "json_object"},
    )
    return res.choices[0].message.content

In [4]:
def persist(data: dict, file):
    """Persist data to the given file."""
    json.dump(data, file, ensure_ascii=False)
    file.write('\n')

In [5]:
def main(query_file_name: str):
    query_file = open(query_file_name, 'r')
    persist_file = open("typo.jsonl", 'a')
    for line in tqdm(query_file):
        temp = json.loads(line)
        queries = {"customer_queries": temp["customer_queries"]}
        typos = typos_augment(queries)
        
        typos = json.loads(typos)
        typos.update({"title": temp["title"], "tone": temp["tone"], "style": "typos"})

        persist(typos, persist_file)

In [6]:
main("tone_augmented.jsonl")
main("queries.jsonl")
main("product_aug.jsonl")

132it [21:35,  9.81s/it]
53it [04:22,  4.96s/it]
13it [03:09, 14.59s/it]


In [7]:
n = 0
with open("typo.jsonl", 'r') as f:
    for json_str in f:
        obj = json.loads(json_str)
        n += len(obj['customer_queries'])

In [8]:
print(n)

3905
